In [1]:
import UDF_registration

In [2]:
import json 

def get_label_pool(sqls):
    pool = {} #attribute -> the set of labels in query workload 
    for sql in sqls:
        filters = sql['filters']
        for left, (op, operand) in filters.items():
            #print(left, operand)
            if left not in pool:
                pool[left] = [operand]
            else:
                pool[left].append(operand)
    #deduplicate labels 
    for attr, labels in pool.items():
        pool[attr] = list(set(labels))
    
    return pool

def write_dict(data, path):
    # Write dictionary to a local file
    with open(path, "w") as file:
        json.dump(data, file, indent=4)

def read_dict(path):
    # Read dictionary from the file
    with open(path, "r") as file:
        loaded_data = json.load(file)
    return loaded_data


In [29]:
#get label pool for paper query workload 
path = '/Users/yiminglin/Documents/Codebase/Text-DB-Share/data/paper/baseline_data/' + 'labels.txt'
sqls = UDF_registration.paper_SQLs()
pool = get_label_pool(sqls)
write_dict(pool, path)
pool = read_dict(path)
pool


{'year': ['2013', '2018', '2017', '2012'],
 'venue': ['CHI', 'CHI or Ubicomp'],
 'author_number': ['5', '3', '4'],
 'contribution': ['empirical'],
 'demographics': ['patients'],
 'study': ['surveys or interviews', 'surveys']}

In [30]:
#get label pool for notice query workload 
path = '/Users/yiminglin/Documents/Codebase/Text-DB-Share/data/NoticeViolation/baseline_data/' + 'labels.txt'
sqls = UDF_registration.notice_SQLs()
pool = get_label_pool(sqls)
write_dict(pool, path)
pool = read_dict(path)
pool


{'region': ['western or central'],
 'penalty': ['10000', '5000', '2000', '0', '20000'],
 'item_num': ['1', '3', '2'],
 'compliance_order': ['false', 'true'],
 'date': ['06/01/2024', '03/01/2024', '01/01/2024'],
 'type': ['Control Room Management or Corrosion Control', 'Corrosion Control'],
 'state': ['California']}

In [31]:
#get label pool for civic query workload 
path = '/Users/yiminglin/Documents/Codebase/Text-DB-Share/data/civic/baseline_data/' + 'labels.txt'
sqls = UDF_registration.civic_SQLs()
pool = get_label_pool(sqls)
write_dict(pool, path)
pool = read_dict(path)
pool


{'type': ['capital', 'disaster'],
 'status': ['not started', 'design'],
 'st': ['2023-01', '2021-05', '2022-06', '2022-01', '2022-03'],
 'et': ['2023-01', '2021-03', '2021-07', '2022-01', '2021-09'],
 'topic': ['road'],
 'ad': ['2021-01']}

In [32]:
#get table values 
prompt_paper = """"
Extract the values for the following attributes from the given document. The attributes are described as follows:

Publication Year: The year in which the work was published. This attribute is a number. 
Type of Contribution: The nature of the work's contribution (e.g., empirical, theoretical, methodological). This attribute is a string. 
Domain: The field or application area the work focuses on. This attribute is a string.  
Type of Study: The research methodology used (e.g., surveys, interviews, experiments). This attribute is a string. 
Venue: The conference or journal where the work was published. This attribute is a string. 
Artifact: Any tool, dataset, framework, or system introduced in the work. This attribute is a string. 
Theory: Any theoretical framework or model proposed or used. This attribute is a string. 
Type of Population Being Studied or Designed: The demographic group the study focuses on (e.g., patients, students, professionals). This attribute is a string. 
Number of Authors: The count of authors who contributed to the work. This attribute is a number. 
Instructions:
Read through the document carefully.
Identify and extract the corresponding values for each attribute.
If any attribute is not explicitly mentioned, indicate it as "Not specified" rather than leaving it blank.
For a row, returns the values for the given attributes in the same order, seperated by |.
If extraction has multiple rows, put each row in a line. Do not add explanations.  
"""

In [5]:
def read_text(file_path):
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        text = file.read()
    return text

def get_schema(dict):
    schema = []
    for attr, desp in dict.items():
        schema.append(attr)
    return schema

def append_line_to_file(file_path, line):
    """
    Appends a single line to the specified file.

    :param file_path: Path to the file.
    :param line: The line to be appended.
    """
    with open(file_path, "a") as file:
        file.write(line + "\n")

def write_text(text, path):
    with open(path, "w") as file:  # "w" mode overwrites existing content
        file.write(text)

import tiktoken

def count_tokens(text, model="gpt-4"):
    """
    Counts the number of tokens in a given text for a specified model.

    :param text: The input text string.
    :param model: The model for which to count tokens (default: "gpt-4").
    :return: The number of tokens in the text.
    """
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

In [35]:
import os, sys  
# Get the directory of the current file
current_file_directory = os.getcwd()
# Get the parent directory
parent_directory = os.path.dirname(current_file_directory)
sys.path.append(parent_directory)
#print(parent_directory)
from model import model 
import time 


#extract paper table data 
model_name = 'gpt4o'

folder_path = "/Users/yiminglin/Documents/Codebase/Text-DB-Share/data/paper/extracted_data"  # Replace with your folder path
out_path = '/Users/yiminglin/Documents/Codebase/Text-DB-Share/data/paper/baseline_data/table.txt'
files = os.listdir(folder_path)
size = 0
st = time.time()
c=0
for file in files: 
    print(c,file)
    path = os.path.join(folder_path, file)
    doc = read_text(path)
    prompt = (prompt_paper, 'This is the document: ' + doc) 
    #print(prompt)
    response = model(model_name,prompt,json=1)
    #print(response)
    size += count_tokens(response)
    size += count_tokens(prompt[0] + prompt[1])
    out = file + ',' + response
    append_line_to_file(out_path, out)
    c+=1
    # if(c>=2):
    #     break

et = time.time()
print(et-st)
print(size)

# paper_schema = get_schema(UDF_registration.paper_attr_desp())
# print(paper_schema)

0 Understanding Animals: A Critical Challenge in ACI.txt
1 Quantifying the Body and Caring for the Mind: Self-Tracking in Multiple Sclerosis.txt
2 Common Barriers to the Use of Patient-Generated Data Across Clinical Settings.txt
3 Eat & Tell: A Randomized Trial of Random-Loss Incentive to Increase Dietary Self-Tracking Compliance.txt
4 The 'Self' As Barrier for Self-Management Technologies in Healthcare?.txt
5 Exploring the Design Space of Glanceable Feedback for Physical Activity Trackers.txt
6 Sundroid: Solar Radiation Awareness with Smartphones.txt
7 No Longer Wearing: Investigating the Abandonment of Personal Health-tracking Technologies on Craigslist.txt
8 What Health Topics Older Adults Want to Track: A Participatory Design Study.txt
9 A Wee Bit More Interaction: Designing and Evaluating an Overactive Bladder App.txt
10 TastyBeats: Designing Palatable Representations of Physical Activity.txt
11 QuittyLink: Using Smartphones for Personal Counseling to Help People Quit Smoking.txt


In [36]:
# extract paper table with label pool

paper_prompt_with_labels = """
Extract the values for the following attributes from the given document. The attributes are described as follows:

Publication Year: The year in which the work was published. This attribute is a number.
Type of Contribution: The nature of the work's contribution. This attribute can only be "empirical". If no match is found, return "Not specified".
Domain: The field or application area the work focuses on. This attribute is a string.
Type of Study: The research methodology used. This attribute can only be "interviews" or "surveys". If no match is found, return "Not specified".
Venue: The conference or journal where the work was published. This attribute is a string.
Artifact: Any tool, dataset, framework, or system introduced in the work. This attribute is a string.
Theory: Any theoretical framework or model proposed or used. This attribute is a string.
Type of Population Being Studied or Designed: The demographic group the study focuses on. This attribute can only be "patients". If no match is found, return "Not specified".
Number of Authors: The count of authors who contributed to the work. This attribute is a number.
Instructions:

Read through the document carefully.
Identify and extract the corresponding values for each attribute.
If any attribute is not explicitly mentioned or does not match the specified values for Contribution, Study, or Demographics, return "Not specified" instead of leaving it blank.
For a row, return the values for the given attributes in the same order, separated by |.
If extraction has multiple rows, put each row on a new line. Do not add explanations.
"""

import os, sys  
# Get the directory of the current file
current_file_directory = os.getcwd()
# Get the parent directory
parent_directory = os.path.dirname(current_file_directory)
sys.path.append(parent_directory)
#print(parent_directory)
from model import model 
import time 


#extract paper table data 
model_name = 'gpt4o'

folder_path = "/Users/yiminglin/Documents/Codebase/Text-DB-Share/data/paper/extracted_data"  # Replace with your folder path
out_path = '/Users/yiminglin/Documents/Codebase/Text-DB-Share/data/paper/baseline_data/table_labels.txt'
files = os.listdir(folder_path)
size = 0
st = time.time()
c=0
for file in files: 
    print(c,file)
    path = os.path.join(folder_path, file)
    doc = read_text(path)
    prompt = (paper_prompt_with_labels, 'This is the document: ' + doc) 
    #print(prompt)
    response = model(model_name,prompt,json=1)
    #print(response)
    size += count_tokens(response)
    size += count_tokens(prompt[0] + prompt[1])
    out = file + '|' + response
    append_line_to_file(out_path, out)
    c+=1
    # if(c>=2):
    #     break

et = time.time()
print(et-st)
print(size)



0 Understanding Animals: A Critical Challenge in ACI.txt
1 Quantifying the Body and Caring for the Mind: Self-Tracking in Multiple Sclerosis.txt
2 Common Barriers to the Use of Patient-Generated Data Across Clinical Settings.txt
3 Eat & Tell: A Randomized Trial of Random-Loss Incentive to Increase Dietary Self-Tracking Compliance.txt
4 The 'Self' As Barrier for Self-Management Technologies in Healthcare?.txt
5 Exploring the Design Space of Glanceable Feedback for Physical Activity Trackers.txt
6 Sundroid: Solar Radiation Awareness with Smartphones.txt
7 No Longer Wearing: Investigating the Abandonment of Personal Health-tracking Technologies on Craigslist.txt
8 What Health Topics Older Adults Want to Track: A Participatory Design Study.txt
9 A Wee Bit More Interaction: Designing and Evaluating an Overactive Bladder App.txt
10 TastyBeats: Designing Palatable Representations of Physical Activity.txt
11 QuittyLink: Using Smartphones for Personal Counseling to Help People Quit Smoking.txt


In [37]:
import os, sys  
# Get the directory of the current file
current_file_directory = os.getcwd()
# Get the parent directory
parent_directory = os.path.dirname(current_file_directory)
sys.path.append(parent_directory)
#print(parent_directory)
from model import model 
import time 

#get table values 
prompt_notice = """"
Extract the values for the following attributes from the given document. The attributes are described as follows:

Name of Company: The name of the company mentioned in the document.
Region of Company: The geographical region where the company operates or is registered.
State of Company: The specific state in which the company is located or mentioned.
Date of Notice: The date when the notice related to the company was issued.
Proposed Civil Penalty: The monetary penalty or fine proposed in the notice.
Type of Violation Item: The nature or category of the violation (e.g., safety, environmental, financial).
Number of Violation Items: The total count of violation items listed in the document.
Compliance Order: Any directives or orders issued to ensure compliance with regulations.
Instructions:

Carefully review the document.
Identify and extract the corresponding values for each attribute.
If any attribute is not explicitly mentioned, indicate it as "Not specified" rather than leaving it blank.
Provide the extracted information in the following format. 
For a row, returns the values for the given attributes in the same order, seperated by |.
If extraction has multiple rows, put each row in a line. Do not add explanations.  
"""

#extract paper table data 
model_name = 'gpt4o'

folder_path = "/Users/yiminglin/Documents/Codebase/Text-DB-Share/data/NoticeViolation/extracted_data"  # Replace with your folder path
out_path = '/Users/yiminglin/Documents/Codebase/Text-DB-Share/data/NoticeViolation/baseline_data/table.txt'
files = os.listdir(folder_path)
size = 0
st = time.time()
c=0
for file in files: 
    print(c,file)
    path = os.path.join(folder_path, file)
    doc = read_text(path)
    prompt = (prompt_notice, 'This is the document: ' + doc) 
    #print(prompt)
    response = model(model_name,prompt,json=1)
    #print(response)
    size += count_tokens(response)
    size += count_tokens(prompt[0] + prompt[1])
    out = file + '|' + response
    append_line_to_file(out_path, out)
    c+=1
    # if(c>=2):
    #     break

et = time.time()
print(et-st)
print(size)

# paper_schema = get_schema(UDF_registration.paper_attr_desp())
# print(paper_schema)

0 32023006NOPV_PCP PCO_05042023_(22-235672)_text.txt
1 12023008NOPV_PCO PCP_06152023_(20-187757)_text.txt
2 42023027NOPV_PCP_03242023_(22-239724)_text.txt
3 52023029NOPV_PCP PCO_06222023_(22-251181)_text.txt
4 52023016NOPV_PCO_03242023_(21-207466)_text.txt
5 12023022NOPV_PCO_04132023_(22-234207)_text.txt
6 32024012NOPV_PCP PCO_02062024_(23-264235)_text.txt
7 42023034NOPV_PCO PCP_05082023_(22-232741)_text.txt
8 52023048NOPV_PCO_11022023_(22-239215)_text.txt
9 52023002NOPV_PCP_02062023_(22-239265)_text.txt
10 32024007NOPV_PCO_01182024_(22-234928)_text.txt
11 12023011NOPV_PCP_05052023_(21-207810)_text.txt
12 52023052NOPV_PCO_10272023_(22-246104)_text.txt
13 32024016NOPV_PCO_02132024_(23-264316)_text.txt
14 12023036NOPV_PCO PCP_05042023_(22-233211)_text.txt
15 32023009NOPV_PCP_05172023_(22-233968)_text.txt
16 52023033NOPV_PCO_07192023_(22-239213)_text.txt
17 52023025NOPV_PCP PCO_04062023_(21-200502)_text.txt
18 42023041NOPV_PCO PCP_03242023_(22-235634)_text.txt
19 32023003NOPV_PCP_08152023

In [38]:
#evaluate notice with labels

import os, sys  
# Get the directory of the current file
current_file_directory = os.getcwd()
# Get the parent directory
parent_directory = os.path.dirname(current_file_directory)
sys.path.append(parent_directory)
#print(parent_directory)
from model import model 
import time 

prompt_notice_with_labels =  """Extract the values for the following attributes from the given document. The attributes are described as follows:

Name of Company: The name of the company mentioned in the document.
Region of Company: The geographical region where the company operates or is registered. This attribute can only be "western" or "central". If no match is found, return "Not specified".
State of Company: The specific state in which the company is located or mentioned. This attribute can only be "California". If no match is found, return "Not specified".
Date of Notice: The date when the notice related to the company was issued.
Proposed Civil Penalty: The monetary penalty or fine proposed in the notice.
Type of Violation Item: The nature or category of the violation. This attribute can only be "Corrosion Control" or "Control Room Management or Corrosion Control". If no match is found, return "Not specified".
Number of Violation Items: The total count of violation items listed in the document.
Compliance Order: Any directives or orders issued to ensure compliance with regulations. This attribute can only be "true" or "false". If no match is found, return "Not specified".
Instructions:

Carefully review the document.
Identify and extract the corresponding values for each attribute.
If any attribute is not explicitly mentioned or does not match the specified values for Region, State, Type of Violation Item, or Compliance Order, return "Not specified" instead of leaving it blank.
Provide the extracted information in the following format:
For a row, return the values for the given attributes in the same order, separated by |.
If extraction has multiple rows, put each row on a new line. Do not add explanations."""

#extract paper table data 
model_name = 'gpt4o'

folder_path = "/Users/yiminglin/Documents/Codebase/Text-DB-Share/data/NoticeViolation/extracted_data"  # Replace with your folder path
out_path = '/Users/yiminglin/Documents/Codebase/Text-DB-Share/data/NoticeViolation/baseline_data/table_labels.txt'
files = os.listdir(folder_path)
size = 0
st = time.time()
c=0
for file in files: 
    print(c,file)
    path = os.path.join(folder_path, file)
    doc = read_text(path)
    prompt = (prompt_notice_with_labels, 'This is the document: ' + doc) 
    #print(prompt)
    response = model(model_name,prompt,json=1)
    #print(response)
    size += count_tokens(response)
    size += count_tokens(prompt[0] + prompt[1])
    out = file + '|' + response
    append_line_to_file(out_path, out)
    c+=1
    # if(c>=2):
    #     break

et = time.time()
print(et-st)
print(size)


0 32023006NOPV_PCP PCO_05042023_(22-235672)_text.txt
1 12023008NOPV_PCO PCP_06152023_(20-187757)_text.txt
2 42023027NOPV_PCP_03242023_(22-239724)_text.txt
3 52023029NOPV_PCP PCO_06222023_(22-251181)_text.txt
4 52023016NOPV_PCO_03242023_(21-207466)_text.txt
5 12023022NOPV_PCO_04132023_(22-234207)_text.txt
6 32024012NOPV_PCP PCO_02062024_(23-264235)_text.txt
7 42023034NOPV_PCO PCP_05082023_(22-232741)_text.txt
8 52023048NOPV_PCO_11022023_(22-239215)_text.txt
9 52023002NOPV_PCP_02062023_(22-239265)_text.txt
10 32024007NOPV_PCO_01182024_(22-234928)_text.txt
11 12023011NOPV_PCP_05052023_(21-207810)_text.txt
12 52023052NOPV_PCO_10272023_(22-246104)_text.txt
13 32024016NOPV_PCO_02132024_(23-264316)_text.txt
14 12023036NOPV_PCO PCP_05042023_(22-233211)_text.txt
15 32023009NOPV_PCP_05172023_(22-233968)_text.txt
16 52023033NOPV_PCO_07192023_(22-239213)_text.txt
17 52023025NOPV_PCP PCO_04062023_(21-200502)_text.txt
18 42023041NOPV_PCO PCP_03242023_(22-235634)_text.txt
19 32023003NOPV_PCP_08152023

In [34]:
import os, sys  
# Get the directory of the current file
current_file_directory = os.getcwd()
# Get the parent directory
parent_directory = os.path.dirname(current_file_directory)
sys.path.append(parent_directory)
#print(parent_directory)
from model import model 
import time 

#get table values 
prompt_civic = """"
Extract the values for the following attributes from the given document. The attributes are described as follows:

The Name of Project: The official name of the project mentioned in the document.
Begin Construction Time: The start date or time when the project construction begins.
Complete Design Time or Completion: The expected or actual completion time of the project design or construction.
Advertise Time: The time when the project was advertised or made public.
Topic: The main subject or focus of the project.
Type: The category or classification of the project.
Status: The current progress or phase of the project (e.g., planned, in progress, completed).
Instructions:

Carefully review the document.
Identify and extract the corresponding values for each attribute.
If any attribute is not explicitly mentioned, indicate it as "Not specified" rather than leaving it blank.
Provide the extracted information in the following format. 
For a row, returns the values for the given attributes in the same order, seperated by comma.
If the extraction result has multiple rows, put each row in a line. Do not add explanations.  
"""

#extract paper table data 
model_name = 'gpt4o'

folder_path = "/Users/yiminglin/Documents/Codebase/Text-DB-Share/data/civic/extracted_data"  # Replace with your folder path

files = os.listdir(folder_path)
size = 0
st = time.time()
c=0
for file in files: 
    print(c,file)
    path = os.path.join(folder_path, file)
    out_path = '/Users/yiminglin/Documents/Codebase/Text-DB-Share/data/civic/baseline_data/' + file 
    doc = read_text(path)
    prompt = (prompt_civic, 'This is the document: ' + doc) 
    #print(prompt)
    response = model(model_name,prompt,json=1)
    #print(response)
    size += count_tokens(response)
    size += count_tokens(prompt[0] + prompt[1])
    write_text(response, out_path)
    c+=1
    # if(c>=2):
    #     break

et = time.time()
print(et-st)
print(size)

# paper_schema = get_schema(UDF_registration.paper_attr_desp())
# print(paper_schema)

0 malibucity_agenda__04282021-1687.txt
1 malibucity_agenda__03232022-1869.txt
2 .DS_Store
3 malibucity_agenda_03222023-2060.txt
4 malibucity_agenda__03022021-1648.txt
5 malibucity_agenda__07272022-1939.txt
6 malibucity_agenda__06222022-1919.txt
7 malibucity_agenda__07282021-1732.txt
8 malibucity_agenda__08252021-1746.txt
9 malibucity_agenda__05262021-1701.txt
10 malibucity_agenda__03242021-1665.txt
11 malibucity_agenda__09222021-1765.txt
12 malibucity_agenda__11102022-1995.txt
13 malibucity_agenda__12142021-1808.txt
14 malibucity_agenda__06282023-2116.txt
15 malibucity_agenda__09272023-2155.txt
16 malibucity_agenda__01272021-1626.txt
17 malibucity_agenda__06232021-1714.txt
18 malibucity_agenda__01262022-1835.txt
19 malibucity_agenda__10272021-1779.txt
352.8707480430603
80530


In [37]:
#evaluate civic with labels 

import os, sys  
# Get the directory of the current file
current_file_directory = os.getcwd()
# Get the parent directory
parent_directory = os.path.dirname(current_file_directory)
sys.path.append(parent_directory)
#print(parent_directory)
from model import model 
import time 

#get table values 
prompt_civic_with_labels = """"
Extract the values for the following attributes from the given document. The attributes are described as follows:

The Name of Project: The official name of the project mentioned in the document.
Begin Construction Time: The start date or time when the project construction begins. This attribute must be in the format YYYY-MM (e.g., "2022-06"). If no match is found, return "Not specified".
Complete Design Time or Completion: The expected or actual completion time of the project design or construction. This attribute must be in the format YYYY-MM (e.g., "2022-06"). If no match is found, return "Not specified".
Advertise Time: The time when the project was advertised or made public. This attribute must be in the format YYYY-MM (e.g., "2022-06"). If no match is found, return "Not specified".
Topic: The main subject or focus of the project. This attribute can only be "road". If no match is found, return "Not specified".
Type: The category or classification of the project. This attribute can only be "capital" or "disaster". If no match is found, return "Not specified".
Status: The current progress or phase of the project. This attribute can only be "design" or "not started". If no match is found, return "Not specified".
Instructions:

Carefully review the document.
Identify and extract the corresponding values for each attribute.
If any attribute is not explicitly mentioned or does not match the specified values for Type, Status, Topic, or Date Fields (st, et, ad), return "Not specified" instead of leaving it blank.
Provide the extracted information in the following format:
For a row, return the values for the given attributes in the same order, separated by commas.
If the extraction result has multiple rows, put each row on a new line. Do not add explanations. 
"""

#extract paper table data 
model_name = 'gpt4o'

folder_path = "/Users/yiminglin/Documents/Codebase/Text-DB-Share/data/civic/extracted_data"  # Replace with your folder path

files = os.listdir(folder_path)
size = 0
st = time.time()
c=0
for file in files: 
    print(c,file)
    path = os.path.join(folder_path, file)
    out_path = '/Users/yiminglin/Documents/Codebase/Text-DB-Share/data/civic/baseline_data/labels_' + file 
    doc = read_text(path)
    prompt = (prompt_civic_with_labels, 'This is the document: ' + doc) 
    #print(prompt)
    response = model(model_name,prompt,json=1)
    #print(response)
    size += count_tokens(response)
    size += count_tokens(prompt[0] + prompt[1])
    write_text(response, out_path)
    c+=1
    # if(c>=2):
    #     break

et = time.time()
print(et-st)
print(size)

# paper_schema = get_schema(UDF_registration.paper_attr_desp())
# print(paper_schema)

0 malibucity_agenda__04282021-1687.txt
1 malibucity_agenda__03232022-1869.txt
2 .DS_Store
3 malibucity_agenda_03222023-2060.txt
4 malibucity_agenda__03022021-1648.txt
5 malibucity_agenda__07272022-1939.txt
6 malibucity_agenda__06222022-1919.txt
7 malibucity_agenda__07282021-1732.txt
8 malibucity_agenda__08252021-1746.txt
9 malibucity_agenda__05262021-1701.txt
10 malibucity_agenda__03242021-1665.txt
11 malibucity_agenda__09222021-1765.txt
12 malibucity_agenda__11102022-1995.txt
13 malibucity_agenda__12142021-1808.txt
14 malibucity_agenda__06282023-2116.txt
15 malibucity_agenda__09272023-2155.txt
16 malibucity_agenda__01272021-1626.txt
17 malibucity_agenda__06232021-1714.txt
18 malibucity_agenda__01262022-1835.txt
19 malibucity_agenda__10272021-1779.txt
319.92205905914307
80951


In [15]:
import re 

def extract_sql_queries(file_path):
    queries = {}
    
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    
    # Updated regex to handle spaces after `#X` and before `SELECT`
    pattern = r'#(\d+)\s*\n\s*(SELECT\b.*?;)'  

    # Use `re.findall()` to capture all matches
    matches = re.findall(pattern, content, re.DOTALL)
    
    # Store in dictionary with integer keys
    for query_id, query in matches:
        queries[int(query_id)] = query.strip()

    return queries

def extract_where_clause(query):
    match = re.search(r'WHERE\s+(.*)', query, re.IGNORECASE)
    return match.group(1) if match else None

def read_and_split_file(file_path):
    vals = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            fields = line.strip().split('|')  # Split by comma
            #print(fields)  # Print each parsed line as a list
            vals.append(fields)
    return vals

def get_kvs(table_vals, table_header):
    kvs = {}
    error = 0
    for vals in table_vals:
        file_name = vals[0]
        #print(len(table_header), len(vals))
        if(len(vals)-len(table_header)!=1):
            error += 1
            continue
        kv = {}
        for i in range(1, len(vals)):
            kv[table_header[i-1]] = vals[i]
        kvs[file_name] = kv
    print(error)
    return kvs

def get_concate_table_row(kv):
    result = ", ".join(f"{k.strip()}: {v.strip()}" for k, v in kv.items())
    return result

def write_2_json(dict, dict_path):
    with open(dict_path, 'w') as file:
        json.dump(dict, file)

In [65]:
prompt = """Prompt:
You are given a predicate and a row of data containing key-value pairs. Your task is to determine whether the row satisfies the predicate.

The predicate is a condition that specifies a comparison between a key and a value.
Only Return a Boolean result (True if the condition is met, False otherwise). Do not add explanations. 
"""

#evaluate query on tables 
query_path = '/Users/yiminglin/Documents/Codebase/Text-DB-Share/query/paper.txt'
queries = extract_sql_queries(query_path)

#get table 
table_path = '/Users/yiminglin/Documents/Codebase/Text-DB-Share/data/paper/baseline_data/table.txt'
table_vals = read_and_split_file(table_path)
#print(table_vals)
#get schema 
schema = get_schema(UDF_registration.paper_attr_desp())
print(schema)

#get table mappings 
table_kvs = get_kvs(table_vals, schema)
#print(table_kvs)



for id, query in queries.items():
    predicates = extract_where_clause(query)
    print(id)
    #scan each document 
    result = {}
    c=0
    for file_name, table_kv in table_kvs.items():
        context = get_concate_table_row(table_kv)
        print('query- ' + str(id) + ' doc- ' + str(c))
        #print(context)
        input = (prompt, 'This is the predicate: ' + predicates + ' . ' + 'This are the row values: ' + context) 
        response = model(model_name, input)
        #print(response)
        if('true' in response.lower()):
            result[file_name] = True
        else:
            result[file_name] = False 
        c+=1
    #write result
    file_name = 'result_text_2_table_without_labels_sql_ver' + str(id) + '.txt'
    write_2_json(result, '/Users/yiminglin/Documents/Codebase/Text-DB-Share/out/paper/' + file_name)





['year', 'contribution', 'domain', 'study', 'venue', 'artifact', 'theory', 'demographics', 'author_number']
0
1
query- 1 doc- 0
query- 1 doc- 1
query- 1 doc- 2
query- 1 doc- 3
query- 1 doc- 4
query- 1 doc- 5
query- 1 doc- 6
query- 1 doc- 7
query- 1 doc- 8
query- 1 doc- 9
query- 1 doc- 10
query- 1 doc- 11
query- 1 doc- 12
query- 1 doc- 13
query- 1 doc- 14
query- 1 doc- 15
query- 1 doc- 16
query- 1 doc- 17
query- 1 doc- 18
query- 1 doc- 19
query- 1 doc- 20
query- 1 doc- 21
query- 1 doc- 22
query- 1 doc- 23
query- 1 doc- 24
query- 1 doc- 25
query- 1 doc- 26
query- 1 doc- 27
query- 1 doc- 28
query- 1 doc- 29
query- 1 doc- 30
query- 1 doc- 31
query- 1 doc- 32
query- 1 doc- 33
query- 1 doc- 34
query- 1 doc- 35
query- 1 doc- 36
query- 1 doc- 37
query- 1 doc- 38
query- 1 doc- 39
query- 1 doc- 40
query- 1 doc- 41
query- 1 doc- 42
query- 1 doc- 43
query- 1 doc- 44
query- 1 doc- 45
query- 1 doc- 46
query- 1 doc- 47
query- 1 doc- 48
query- 1 doc- 49
query- 1 doc- 50
query- 1 doc- 51
query- 1 doc- 

In [66]:
prompt = """Prompt:
You are given a predicate and a row of data containing key-value pairs. Your task is to determine whether the row satisfies the predicate.

The predicate is a condition that specifies a comparison between a key and a value.
Only Return a Boolean result (True if the condition is met, False otherwise). Do not add explanations. 
"""

#evaluate query on tables 
query_path = '/Users/yiminglin/Documents/Codebase/Text-DB-Share/query/paper.txt'
queries = extract_sql_queries(query_path)

#get table 
table_path = '/Users/yiminglin/Documents/Codebase/Text-DB-Share/data/paper/baseline_data/table_labels.txt'
table_vals = read_and_split_file(table_path)
#print(table_vals)
#get schema 
schema = get_schema(UDF_registration.paper_attr_desp())
print(schema)

#get table mappings 
table_kvs = get_kvs(table_vals, schema)
#print(table_kvs)



for id, query in queries.items():
    predicates = extract_where_clause(query)
    print(id)
    #scan each document 
    result = {}
    c=0
    for file_name, table_kv in table_kvs.items():
        context = get_concate_table_row(table_kv)
        print('query- ' + str(id) + ' doc- ' + str(c))
        #print(context)
        input = (prompt, 'This is the predicate: ' + predicates + ' . ' + 'This are the row values: ' + context) 
        response = model(model_name, input)
        #print(response)
        if('true' in response.lower()):
            result[file_name] = True
        else:
            result[file_name] = False 
        c+=1
    #write result
    file_name = 'result_text_2_table_with_labels_sql_ver' + str(id) + '.txt'
    write_2_json(result, '/Users/yiminglin/Documents/Codebase/Text-DB-Share/out/paper/' + file_name)





['year', 'contribution', 'domain', 'study', 'venue', 'artifact', 'theory', 'demographics', 'author_number']
0
1
query- 1 doc- 0
query- 1 doc- 1
query- 1 doc- 2
query- 1 doc- 3
query- 1 doc- 4
query- 1 doc- 5
query- 1 doc- 6
query- 1 doc- 7
query- 1 doc- 8
query- 1 doc- 9
query- 1 doc- 10
query- 1 doc- 11
query- 1 doc- 12
query- 1 doc- 13
query- 1 doc- 14
query- 1 doc- 15
query- 1 doc- 16
query- 1 doc- 17
query- 1 doc- 18
query- 1 doc- 19
query- 1 doc- 20
query- 1 doc- 21
query- 1 doc- 22
query- 1 doc- 23
query- 1 doc- 24
query- 1 doc- 25
query- 1 doc- 26
query- 1 doc- 27
query- 1 doc- 28
query- 1 doc- 29
query- 1 doc- 30
query- 1 doc- 31
query- 1 doc- 32
query- 1 doc- 33
query- 1 doc- 34
query- 1 doc- 35
query- 1 doc- 36
query- 1 doc- 37
query- 1 doc- 38
query- 1 doc- 39
query- 1 doc- 40
query- 1 doc- 41
query- 1 doc- 42
query- 1 doc- 43
query- 1 doc- 44
query- 1 doc- 45
query- 1 doc- 46
query- 1 doc- 47
query- 1 doc- 48
query- 1 doc- 49
query- 1 doc- 50
query- 1 doc- 51
query- 1 doc- 

In [19]:
prompt = """Prompt:
You are given a predicate and a row of data containing key-value pairs. Your task is to determine whether the row satisfies the predicate.

The predicate is a condition that specifies a comparison between a key and a value.
Only Return a Boolean result (True if the condition is met, False otherwise). Do not add explanations. 
"""

import os, sys  
# Get the directory of the current file
current_file_directory = os.getcwd()
# Get the parent directory
parent_directory = os.path.dirname(current_file_directory)
sys.path.append(parent_directory)
#print(parent_directory)
from model import model 
model_name = 'gpt4o'
#evaluate query on tables 
query_path = '/Users/yiminglin/Documents/Codebase/Text-DB-Share/query/NoticeViolation.txt'
queries = extract_sql_queries(query_path)
#print(queries)
#get table 
table_path = '/Users/yiminglin/Documents/Codebase/Text-DB-Share/data/NoticeViolation/baseline_data/table.txt'
table_vals = read_and_split_file(table_path)
#print(table_vals)
#get schema 
schema = get_schema(UDF_registration.notice_attr_desp())
print(schema)

#get table mappings 
table_kvs = get_kvs(table_vals, schema)
print(table_kvs)



for id, query in queries.items():
    predicates = extract_where_clause(query)
    print(id)
    #scan each document 
    result = {}
    c=0
    for file_name, table_kv in table_kvs.items():
        context = get_concate_table_row(table_kv)
        print('query- ' + str(id) + ' doc- ' + str(c))
        #print(context)
        input = (prompt, 'This is the predicate: ' + predicates + ' . ' + 'This are the row values: ' + context) 
        response = model(model_name, input)
        #print(response)
        if('true' in response.lower()):
            result[file_name] = True
        else:
            result[file_name] = False 
        c+=1
    #write result
    file_name = 'result_text_2_table_without_labels_sql_ver' + str(id) + '.txt'
    write_2_json(result, '/Users/yiminglin/Documents/Codebase/Text-DB-Share/out/NoticeViolation/' + file_name)





['company', 'region', 'state', 'date', 'penalty', 'type', 'item_num', 'compliance_order']
0
{'32023006NOPV_PCP PCO_05042023_(22-235672)_text.txt': {'company': 'Coffeyville Resources Crude Transportation, LLC ', 'region': ' Central Region ', 'state': ' Oklahoma ', 'date': ' May 4, 2023 ', 'penalty': ' $804,300 ', 'type': ' Safety ', 'item_num': ' 1 ', 'compliance_order': ' CRCT must, within 90 days of the Final Order, provide an updated internal corrosion monitoring procedure that includes a process to periodically survey its pipeline segments for dead leg segments.'}, '12023008NOPV_PCO PCP_06152023_(20-187757)_text.txt': {'company': 'Adelphia Gateway, LLC ', 'region': ' Not specified ', 'state': ' New Jersey ', 'date': ' June 15, 2023 ', 'penalty': ' $56,600 ', 'type': ' Safety ', 'item_num': ' 1 ', 'compliance_order': ' Install proper line markers in accordance with 49 C.F.R. § 192.707 at its Perkiomen and East Perkiomen MLV sites, and at all other aboveground natural gas facilities i

In [20]:
prompt = """Prompt:
You are given a predicate and a row of data containing key-value pairs. Your task is to determine whether the row satisfies the predicate.

The predicate is a condition that specifies a comparison between a key and a value.
Only Return a Boolean result (True if the condition is met, False otherwise). Do not add explanations. 
"""

import os, sys  
# Get the directory of the current file
current_file_directory = os.getcwd()
# Get the parent directory
parent_directory = os.path.dirname(current_file_directory)
sys.path.append(parent_directory)
#print(parent_directory)
from model import model 
model_name = 'gpt4o'
#evaluate query on tables 
query_path = '/Users/yiminglin/Documents/Codebase/Text-DB-Share/query/NoticeViolation.txt'
queries = extract_sql_queries(query_path)
#print(queries)
#get table 
table_path = '/Users/yiminglin/Documents/Codebase/Text-DB-Share/data/NoticeViolation/baseline_data/table_labels.txt'
table_vals = read_and_split_file(table_path)
#print(table_vals)
#get schema 
schema = get_schema(UDF_registration.notice_attr_desp())
print(schema)

#get table mappings 
table_kvs = get_kvs(table_vals, schema)
print(table_kvs)



for id, query in queries.items():
    predicates = extract_where_clause(query)
    print(id)
    #scan each document 
    result = {}
    c=0
    for file_name, table_kv in table_kvs.items():
        context = get_concate_table_row(table_kv)
        print('query- ' + str(id) + ' doc- ' + str(c))
        #print(context)
        input = (prompt, 'This is the predicate: ' + predicates + ' . ' + 'This are the row values: ' + context) 
        response = model(model_name, input)
        #print(response)
        if('true' in response.lower()):
            result[file_name] = True
        else:
            result[file_name] = False 
        c+=1
    #write result
    file_name = 'result_text_2_table_with_labels_sql_ver' + str(id) + '.txt'
    write_2_json(result, '/Users/yiminglin/Documents/Codebase/Text-DB-Share/out/NoticeViolation/' + file_name)





['company', 'region', 'state', 'date', 'penalty', 'type', 'item_num', 'compliance_order']
8
{'32023006NOPV_PCP PCO_05042023_(22-235672)_text.txt': {'company': 'Coffeyville Resources Crude Transportation, LLC', 'region': 'central', 'state': 'Not specified', 'date': 'May 4, 2023', 'penalty': '$804,300', 'type': 'Corrosion Control', 'item_num': '1', 'compliance_order': 'true'}, '12023008NOPV_PCO PCP_06152023_(20-187757)_text.txt': {'company': 'Adelphia Gateway, LLC', 'region': 'Not specified', 'state': 'Not specified', 'date': 'June 15, 2023', 'penalty': '$56,600', 'type': 'Not specified', 'item_num': '1', 'compliance_order': 'true'}, '42023027NOPV_PCP_03242023_(22-239724)_text.txt': {'company': 'Sea Robin Pipeline Company', 'region': 'central', 'state': 'Not specified', 'date': 'March 24, 2023', 'penalty': '$39,400', 'type': 'Corrosion Control', 'item_num': '1', 'compliance_order': 'Not specified'}, '52023029NOPV_PCP PCO_06222023_(22-251181)_text.txt': {'company': 'AmeriGas Propane LP', 

In [32]:
#evaluate final results 

def get_title(name):
    match = re.match(r"(\d+)", name)

    if match:
        extracted_name = f"{match.group(1)}.txt"
        return extracted_name

def eval(results, truths, data): 
    matched = 0
    for f, result in results.items():
        if(data == 'NoticeViolation'):
            f = get_title(f)
        truth = truths[f]
        if(result == truth):
            matched += 1
    precision = matched/len(results)
    recall = matched/len(truths)
    return precision, recall

In [29]:
avg_precision = 0
avg_recall = 0

c=0
for i in range(1,22):
    #print(i)
    
    result_path = '/Users/yiminglin/Documents/Codebase/Text-DB-Share/out/paper/result_text_2_table_with_labels_sql_ver' + str(i) + '.txt'
    results = read_dict(result_path)
    truth_path = '/Users/yiminglin/Documents/Codebase/TextDB/Text-DB/data/paper/labeled_data/large_truth_sql_ver' + str(i) + '.txt'
    truths = read_dict(truth_path)
    precision, recall = eval(results, truths)
    #print(precision, recall)

    avg_precision += precision
    avg_recall += recall
    c+=1

avg_precision /= c
avg_recall /= c

print(avg_precision, avg_recall)


0.7542087542087542 0.2882882882882883


In [30]:
avg_precision = 0
avg_recall = 0

c=0
for i in range(1,22):
    #print(i)
    
    result_path = '/Users/yiminglin/Documents/Codebase/Text-DB-Share/out/paper/result_text_2_table_without_labels_sql_ver' + str(i) + '.txt'
    results = read_dict(result_path)
    truth_path = '/Users/yiminglin/Documents/Codebase/TextDB/Text-DB/data/paper/labeled_data/large_truth_sql_ver' + str(i) + '.txt'
    truths = read_dict(truth_path)
    precision, recall = eval(results, truths)
    #print(precision, recall)

    avg_precision += precision
    avg_recall += recall
    c+=1

avg_precision /= c
avg_recall /= c

print(avg_precision, avg_recall)


0.7431457431457432 0.2840595697738555


In [33]:
avg_precision = 0
avg_recall = 0

data = 'NoticeViolation'
c=0
for i in range(1,22):
    #print(i)
    
    result_path = '/Users/yiminglin/Documents/Codebase/Text-DB-Share/out/NoticeViolation/result_text_2_table_with_labels_sql_ver' + str(i) + '.txt'
    results = read_dict(result_path)
    truth_path = '/Users/yiminglin/Documents/Codebase/TextDB/Text-DB/data/NoticeViolation/labeled_data/large_truth_sql_ver' + str(i) + '.txt'
    truths = read_dict(truth_path)
    precision, recall = eval(results, truths, data)
    #print(precision, recall)

    avg_precision += precision
    avg_recall += recall
    c+=1

avg_precision /= c
avg_recall /= c

print(avg_precision, avg_recall)


0.837251356238698 0.8267857142857143


In [34]:
avg_precision = 0
avg_recall = 0

data = 'NoticeViolation'
c=0
for i in range(1,22):
    #print(i)
    
    result_path = '/Users/yiminglin/Documents/Codebase/Text-DB-Share/out/NoticeViolation/result_text_2_table_without_labels_sql_ver' + str(i) + '.txt'
    results = read_dict(result_path)
    truth_path = '/Users/yiminglin/Documents/Codebase/TextDB/Text-DB/data/NoticeViolation/labeled_data/large_truth_sql_ver' + str(i) + '.txt'
    truths = read_dict(truth_path)
    precision, recall = eval(results, truths, data)
    #print(precision, recall)

    avg_precision += precision
    avg_recall += recall
    c+=1

avg_precision /= c
avg_recall /= c

print(avg_precision, avg_recall)


0.8214285714285716 0.8214285714285716
